In [3]:
import os
import pandas as pd
import numpy as np
import psycopg2
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

/Users/dom/Desktop/Thesis/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
file_path = os.path.join(os.getcwd(), 'Model_metadata.xlsx')  
df = pd.read_excel(file_path)
df.info()
#df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Input Data                        263 non-null    object 
 1   Action                            156 non-null    object 
 2   Model Task                        262 non-null    object 
 3   Model                             261 non-null    object 
 4   Framework/Library                 127 non-null    object 
 5   Github 
stars(*1000)              126 non-null    float64
 6   Normalized Github 
stars(*1000)   127 non-null    float64
 7   Citations                         126 non-null    float64
 8   Normalized Citations              127 non-null    float64
 9   Open Source/Proprietary           123 non-null    object 
 10  License                           124 non-null    object 
 11  Model Size(MB)                    124 non-null    object 
 12  Model Si

In [19]:
#Data Cleaning

# Fill missing values with appropriate placeholders
df.fillna({
    'Github stars': 0,
    'Citations': 0,
    'Model Size (MB)': 'Unknown',
    'Memory Requirement (training)': 'Unknown',
    'License': 'Unknown',
    'URL': 'Unknown',
    'Paper link': 'Unknown',
    'Hardware Accelerators': 'Unknown',
    'Detailed Documentation?': 'Unknown',
    'Documentation Link': 'Unknown',
    'Repository Link': 'Unknown',
    'Paper Link': 'Unknown'
}, inplace=True)


In [20]:
# Check for null values in each column
null_values = df.isnull().sum()
print(null_values)

Input Data                            2
Action                              109
Model Task                            3
Model                                 4
Framework/Library                   138
Github \nstars(*1000)               139
Normalized Github \nstars(*1000)    138
Citations                             0
Normalized Citations                138
Open Source/Proprietary             142
License                               0
Model Size(MB)                      141
Model Size Normalized(MB)           143
Memory Requirement(training)        139
Memory Requirement Numeric          139
Memory Requirement Normalized       139
Hardware Aceelerators               139
Hardware Aceelerators Normalized    139
Minimum CPU/GPU/TPU Normalized      140
Detailed \nDocumentation?           140
Institution                         136
Institution Numeric                 140
URL                                   0
Paper link                            0
dtype: int64


In [ ]:
df.rename(columns={
    'Detailed \nDocumentation?': 'Detailed Documentation',
    'Minimum   CPU/GPU/TPU Normalized': 'Minimum Hardware Accelerators',
    'Normalized Github \nstars(*1000)': 'Normalized Github stars',
    'Institution Numeric': 'Institution Encoded',
    'Hardware Aceelerators Normalized': 'Hardware Accelerators Normalized',
    'Memory Requirement \nNormalized': 'Memory Requirement Normalized',
}, inplace=True)

In [30]:

#Feature Engineering 
# Normalize selected numeric fields
#def normalize_column(col):
 #   return (col - col.min()) / (col.max() - col.min())

#numeric_cols = ['Github stars', 'Citations', 'Inference Time(s)', 'Model Size (MB)']
#for col in numeric_cols:
 #   if col in df.columns:
  #      df[f'Normalized {col}'] = normalize_column(df[col])

df['Detailed Documentation Encoded'] = df['Detailed Documentation'].astype(str).str.strip().str.upper().apply(
    lambda x: 1 if x.startswith('YES') else 0
)


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Input Data                        263 non-null    object 
 1   Action                            156 non-null    object 
 2   Model Task                        262 non-null    object 
 3   Model                             261 non-null    object 
 4   Framework/Library                 127 non-null    object 
 5   Github 
stars(*1000)              126 non-null    float64
 6   Normalized Github stars           127 non-null    float64
 7   Citations                         265 non-null    float64
 8   Normalized Citations              127 non-null    float64
 9   Open Source/Proprietary           123 non-null    object 
 10  License                           265 non-null    object 
 11  Model Size(MB)                    124 non-null    object 
 12  Model Si

In [32]:
df.head(10)

,Input Data,Action,Model Task,Model,Framework/Library,Github \nstars(*1000),Normalized Github stars,Citations,Normalized Citations,Open Source/Proprietary,...,Memory Requirement Normalized,Hardware Aceelerators,Hardware Aceelerators Normalized,Normalized Hardware Aceelerators,Detailed Documentation,Institution,Institution Encoded,URL,Paper link,Detailed Documentation Encoded
0,Text,Identify and classify named entities such as ...,Named Entity Recognition,Bidirectional-LSTM,Tensorflow,0.35,0.004743,1678.0,0.016700,Open source,...,0.066667,GPU,0.67,1.0,Yes,Kamal Kraj,0.0,https://github.com/kamalkraj/Named-Entity-Reco...,https://direct.mit.edu/tacl/article-abstract/d...,1
1,Text,Identify and classify named entities such as ...,Named Entity Recognition,BERT,Tensorflow,36.40,0.493225,41077.0,0.408808,Open source,...,0.466667,GPU/TPU,1.00,1.0,Yes,Google,1.0,https://github.com/google-research/bert.git,https://arxiv.org/abs/1810.04805,1
2,Text,Identify and classify named entities such as ...,Named Entity Recognition,Flair,PyTorch,13.40,0.181572,10182.0,0.101334,Open source,...,0.200000,GPU,0.67,1.0,Yes,Flair,1.0,https://github.com/flairNLP/flair.git,https://www.sciencedirect.com/science/article/...,1
3,Text,Identify and classify named entities such as ...,Named Entity Recognition,Sequence_tagging,Tensorflow,1.90,0.025745,2509.0,0.024970,Open source,...,0.066667,GPU,0.67,1.0,No,Guillaume Genthial,0.0,https://github.com/guillaumegenthial/sequence_...,1603.01354v5.pdf (arxiv.org),0
4,Text,Identify and classify named entities such as ...,Named Entity Recognition,gluon_nlp,mxnet,2.60,0.035230,10232.0,0.101831,Open source,...,0.200000,GPU,0.67,1.0,Yes,Gluon,1.0,https://github.com/dmlc/gluon-nlp.git,https://www.sciencedirect.com/science/article/...,1
5,Text,Identify and classify named entities such as ...,Named Entity Recognition,tf_ner,Tensorflow,0.93,0.012602,2524.0,0.025119,Open source,...,0.066667,GPU,0.67,1.0,No,Guillaume Genthial,0.0,https://github.com/guillaumegenthial/tf_ner.git,https://arxiv.org/abs/1603.01354,0
6,Text,Identify and classify named entities such as ...,Named Entity Recognition,ERNIE,Paddle,6.10,0.082656,475.0,0.004727,Open source,...,0.466667,GPU,0.67,1.0,Yes,n.a,0.0,https://github.com/PaddlePaddle/ERNIE.git,https://arxiv.org/abs/1904.09223,1
7,Text,NaN,Feature Extraction,nvidia/NV-Embed-v2,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/nvidia/NV-Embed-v2,Unknown,0
8,Text,NaN,Feature Extraction,intfloat/multilingual-e5-large,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/intfloat/multilingual-e...,Unknown,0
9,Text,NaN,Feature Extraction,canine-s,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,google,NaN,https://huggingface.co/google/canine-s,Unknown,0


In [ ]:
output_path = '/Users/dom/Desktop/Thesis/thesis_root/data/prepropessed data/model_metadata.csv'
df.to_csv(output_path, index=False)

In [ ]:
file_path = os.path.join(os.getcwd(), 'prepropessed data/model_metadata.csv')  
df = pd.read_csv(file_path)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Input Data                        263 non-null    object 
 1   Action                            156 non-null    object 
 2   Model Task                        262 non-null    object 
 3   Model                             261 non-null    object 
 4   Framework/Library                 127 non-null    object 
 5   Github 
stars(*1000)              126 non-null    float64
 6   Normalized Github stars           127 non-null    float64
 7   Citations                         265 non-null    float64
 8   Normalized Citations              127 non-null    float64
 9   Open Source/Proprietary           123 non-null    object 
 10  License                           265 non-null    object 
 11  Model Size(MB)                    124 non-null    object 
 12  Model Si

In [9]:
df.head(10)

,Input Data,Action,Model Task,Model,Framework/Library,Github \nstars(*1000),Normalized Github stars,Citations,Normalized Citations,Open Source/Proprietary,...,Memory Requirement Normalized,Hardware Aceelerators,Hardware Aceelerators Normalized,Minumum Hardware Aceelerators,Detailed Documentation,Institution,Institution Encoded,URL,Paper link,Detailed Documentation Encoded
0,Text,Identify and classify named entities such as ...,Named Entity Recognition,Bidirectional-LSTM,Tensorflow,0.35,0.004743,1678.0,0.016700,Open source,...,0.066667,GPU,0.67,1.0,Yes,Kamal Kraj,0.0,https://github.com/kamalkraj/Named-Entity-Reco...,https://direct.mit.edu/tacl/article-abstract/d...,1
1,Text,Identify and classify named entities such as ...,Named Entity Recognition,BERT,Tensorflow,36.40,0.493225,41077.0,0.408808,Open source,...,0.466667,GPU/TPU,1.00,1.0,Yes,Google,1.0,https://github.com/google-research/bert.git,https://arxiv.org/abs/1810.04805,1
2,Text,Identify and classify named entities such as ...,Named Entity Recognition,Flair,PyTorch,13.40,0.181572,10182.0,0.101334,Open source,...,0.200000,GPU,0.67,1.0,Yes,Flair,1.0,https://github.com/flairNLP/flair.git,https://www.sciencedirect.com/science/article/...,1
3,Text,Identify and classify named entities such as ...,Named Entity Recognition,Sequence_tagging,Tensorflow,1.90,0.025745,2509.0,0.024970,Open source,...,0.066667,GPU,0.67,1.0,No,Guillaume Genthial,0.0,https://github.com/guillaumegenthial/sequence_...,1603.01354v5.pdf (arxiv.org),0
4,Text,Identify and classify named entities such as ...,Named Entity Recognition,gluon_nlp,mxnet,2.60,0.035230,10232.0,0.101831,Open source,...,0.200000,GPU,0.67,1.0,Yes,Gluon,1.0,https://github.com/dmlc/gluon-nlp.git,https://www.sciencedirect.com/science/article/...,1
5,Text,Identify and classify named entities such as ...,Named Entity Recognition,tf_ner,Tensorflow,0.93,0.012602,2524.0,0.025119,Open source,...,0.066667,GPU,0.67,1.0,No,Guillaume Genthial,0.0,https://github.com/guillaumegenthial/tf_ner.git,https://arxiv.org/abs/1603.01354,0
6,Text,Identify and classify named entities such as ...,Named Entity Recognition,ERNIE,Paddle,6.10,0.082656,475.0,0.004727,Open source,...,0.466667,GPU,0.67,1.0,Yes,n.a,0.0,https://github.com/PaddlePaddle/ERNIE.git,https://arxiv.org/abs/1904.09223,1
7,Text,NaN,Feature Extraction,nvidia/NV-Embed-v2,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/nvidia/NV-Embed-v2,Unknown,0
8,Text,NaN,Feature Extraction,intfloat/multilingual-e5-large,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/intfloat/multilingual-e...,Unknown,0
9,Text,NaN,Feature Extraction,canine-s,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,google,NaN,https://huggingface.co/google/canine-s,Unknown,0


In [ ]:
#df.rename(columns={

 #   'Normalized Hardware Aceelerators': 'Minumum Hardware Aceelerators',

#}, inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Input Data                        263 non-null    object 
 1   Action                            156 non-null    object 
 2   Model Task                        262 non-null    object 
 3   Model                             261 non-null    object 
 4   Framework/Library                 127 non-null    object 
 5   Github 
stars(*1000)              126 non-null    float64
 6   Normalized Github stars           127 non-null    float64
 7   Citations                         265 non-null    float64
 8   Normalized Citations              127 non-null    float64
 9   Open Source/Proprietary           123 non-null    object 
 10  License                           265 non-null    object 
 11  Model Size(MB)                    124 non-null    object 
 12  Model Si

In [38]:
dflist = df.columns.tolist()
print(dflist)

['Input Data', 'Action', 'Model Task', 'Model', 'Framework/Library', 'Github \nstars(*1000)', 'Normalized Github stars', 'Citations', 'Normalized Citations', 'Open Source/Proprietary', 'License', 'Model Size(MB)', 'Model Size Normalized(MB)', 'Memory Requirement(training)', 'Memory Requirement Numeric', 'Memory Requirement Normalized  ', 'Hardware Aceelerators', 'Hardware Accelerators Normalized', 'Minumum Hardware Aceelerators', 'Detailed Documentation', 'Institution', 'Institution Encoded', 'URL', 'Paper link', 'Detailed Documentation Encoded', 'Overall_Score', 'Performance_Score', 'Popularity_Score', 'Licensing_Score', 'Hardware_Score', 'Documentation_Score', 'Institution_Score']


## Scoring Engine

In [30]:
def calculate_performance_score(df):
    """
    Calculates the performance score based only on normalized model size and memory usage.
    Lower model size and lower memory usage = higher performance score.
    """
    return (1 - df['Model Size Normalized(MB)']) * 0.5 + (1 - df['Memory Requirement Normalized  ']) * 0.5

def calculate_popularity_score(df):
    """
    Calculates popularity score based on GitHub stars and Citations.
    Higher stars and citations = higher score.
    """
    return df['Normalized Github stars'] * 0.6 + df['Normalized Citations'] * 0.4

def calculate_licensing_score(df):
    """
    Assigns higher score to open-source models because of flexibility and lower cost.
    """
    return df['Open Source/Proprietary'].apply(lambda x: 1.0 if x == "Open Source" else 0.5)

def calculate_hardware_score(df):
    """
    Gives higher score to models that are compatible with CPU (universally available) and optimized GPU/TPU usage.
    """
    return df['Hardware Aceelerators Normalized'] * 0.5 + (1 - df['Minumum Hardware Aceelerators']) * 0.5

def calculate_documentation_score(df):
    """
    Higher score if detailed documentation exists. Documentation ensures faster and successful deployment.
    """
    return df['Detailed Documentation Encoded'].apply(lambda x: 1.0 if x == "Yes" else 0.5)

def calculate_institution_score(df):
    """
    Models from top institutions (Google, OpenAI, Facebook) might be prioritized for credibility and maintenance.
    """
    return df['Institution Encoded']


def overall_model_score(df, weights=None):
    """
    Aggregates all individual scores into one overall model score and includes each score in the dataframe.
    """
    if weights is None:
        weights = {
            'performance': 0.30,
            'popularity': 0.20,
            'licensing': 0.15,
            'hardware': 0.15,
            'documentation': 0.10,
            'institution': 0.10
        }
    
    df['Performance_Score'] = calculate_performance_score(df)
    df['Popularity_Score'] = calculate_popularity_score(df)
    df['Licensing_Score'] = calculate_licensing_score(df)
    df['Hardware_Score'] = calculate_hardware_score(df)
    df['Documentation_Score'] = calculate_documentation_score(df)
    df['Institution_Score'] = calculate_institution_score(df)
    
    df['Overall_Score'] = (
        df['Performance_Score'] * weights['performance'] +
        df['Popularity_Score'] * weights['popularity'] +
        df['Licensing_Score'] * weights['licensing'] +
        df['Hardware_Score'] * weights['hardware'] +
        df['Documentation_Score'] * weights['documentation'] +
        df['Institution_Score'] * weights['institution']
    )
    return df

# Example usage:
# df_scored = overall_model_score(df)
# df_scored.sort_values(by='Overall_Score', ascending=False).head(10)


In [31]:
df_scored = overall_model_score(df)
df_scored.sort_values(by='Overall_Score', ascending=False).head(10)

,Input Data,Action,Model Task,Model,Framework/Library,Github \nstars(*1000),Normalized Github stars,Citations,Normalized Citations,Open Source/Proprietary,...,URL,Paper link,Detailed Documentation Encoded,Overall_Score,Performance_Score,Popularity_Score,Licensing_Score,Hardware_Score,Documentation_Score,Institution_Score
77,Text,translate the input text into another target l...,Machine translation,transformers,PyTorch,66.7,0.903794,2600.0,0.025876,Open Source,...,https://github.com/huggingface/transformers.git,https://arxiv.org/abs/1907.06616,1,0.676997,0.720739,0.552627,1.0,0.335,0.5,1.0
67,Text,translate the input text into another target l...,Machine translation,Attention is all you need,Tensorflow,73.8,1.000000,43574.0,0.433658,Open Source,...,tensorflow/tensor2tensor: Library of deep lear...,[1706.03762] Attention Is All You Need (arxiv....,1,0.674511,0.482728,0.773463,1.0,0.500,0.5,1.0
143,Video/Images,estimate the pose or body positioning of peopl...,Pose estimation,Detectron 2,PyTorch,21.2,0.287263,13858.0,0.137918,Open Source,...,facebookresearch/detectron2: Detectron2 is a p...,https://arxiv.org/abs/1708.02002,1,0.664880,0.897083,0.227525,1.0,0.335,0.5,1.0
68,Text,translate the input text into another target l...,Machine translation,Tensor2tensor,Tensorflow,12.4,0.168022,43574.0,0.433658,Open Source,...,tensorflow/tensor2tensor: Library of deep lear...,[1706.03762] Attention Is All You Need (arxiv....,1,0.657434,0.758597,0.274276,1.0,0.500,0.5,1.0
74,Text,translate the input text into another target l...,Machine translation,Tensorpack,Tensorflow,6.2,0.084011,13737.0,0.136714,Open Source,...,tensorpack/tensorpack: A Neural Net Training I...,Unknown,1,0.639968,0.895666,0.105092,1.0,0.335,0.5,1.0
54,Text,"generate new text based on the input, such as ...",Text Generation,Transformers,PyTorch,65.3,0.884824,46166.0,0.459455,Open source,...,huggingface/transformers: 🤗 Transformers: Stat...,https://arxiv.org/pdf/1910.03771v5.pdf,1,0.634623,0.721460,0.714676,0.5,0.335,0.5,1.0
78,Text,translate the input text into another target l...,Machine translation,MUSE,PyTorch,3.0,0.040650,1295.0,0.012888,Open Source,...,https://github.com/facebookresearch/MUSE.git,https://arxiv.org/abs/1710.04087,1,0.626128,0.899897,0.029545,1.0,0.335,0.5,1.0
69,Text,translate the input text into another target l...,Machine translation,PyTorch-Seq2Seq,PyTorch,1.4,0.018970,2886.0,0.028722,Open Source,...,IBM/pytorch-seq2seq: An open source framework ...,https://arxiv.org/abs/1705.03122,1,0.623625,0.896003,0.022871,1.0,0.335,0.5,1.0
1,Text,Identify and classify named entities such as ...,Named Entity Recognition,BERT,Tensorflow,36.4,0.493225,41077.0,0.408808,Open source,...,https://github.com/google-research/bert.git,https://arxiv.org/abs/1810.04805,1,0.621846,0.766515,0.459458,0.5,0.500,0.5,1.0
131,Video/Images,detect and localize objects of interest within...,Object detection,Darknet,Tensorflow,19.2,0.260163,24875.0,0.247562,Open source,...,https://github.com/AlexeyAB/darknet.git,https://www.cv-foundation.org/openaccess/conte...,1,0.614209,0.959783,0.255122,0.5,0.335,0.5,1.0


In [44]:
output_path = '/Users/dom/Desktop/Thesis/thesis_root/data/prepropessed data/model_metadata_final.csv'
df.to_csv(output_path, index=False)


In [45]:
file_path = os.path.join(os.getcwd(), 'prepropessed data/model_metadata_final.csv')  
df = pd.read_csv(file_path)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Input Data                        263 non-null    object 
 1   Action                            156 non-null    object 
 2   Model Task                        262 non-null    object 
 3   Model                             261 non-null    object 
 4   Framework/Library                 127 non-null    object 
 5   Github stars                      126 non-null    float64
 6   Normalized Github stars           127 non-null    float64
 7   Citations                         265 non-null    float64
 8   Normalized Citations              127 non-null    float64
 9   Open Source/Proprietary           123 non-null    object 
 10  License                           265 non-null    object 
 11  Model Size(MB)                    124 non-null    object 
 12  Model Si

In [ ]:
df.rename(columns={
    'Detailed \nDocumentation?': 'Detailed Documentation',
    'Minumum Hardware Aceelerators': 'Minimum Hardware Accelerators',
    'Github \nstars(*1000)': 'Github stars',
    'Institution Numeric': 'Institution Encoded',
    'Hardware Aceelerators Normalized': 'Hardware Accelerators Normalized',
    'Memory Requirement Normalized  ': 'Memory Requirement Normalized',
}, inplace=True)

In [42]:
df.rename(columns={
    'Hardware Aceelerators': 'Hardware Accelerators',
}, inplace=True)

In [43]:
df.columns.tolist()

['Input Data',
 'Action',
 'Model Task',
 'Model',
 'Framework/Library',
 'Github stars',
 'Normalized Github stars',
 'Citations',
 'Normalized Citations',
 'Open Source/Proprietary',
 'License',
 'Model Size(MB)',
 'Model Size Normalized(MB)',
 'Memory Requirement(training)',
 'Memory Requirement Numeric',
 'Memory Requirement Normalized',
 'Hardware Accelerators',
 'Hardware Accelerators Normalized',
 'Minimum Hardware Accelerators',
 'Detailed Documentation',
 'Institution',
 'Institution Encoded',
 'URL',
 'Paper link',
 'Detailed Documentation Encoded',
 'Overall_Score',
 'Performance_Score',
 'Popularity_Score',
 'Licensing_Score',
 'Hardware_Score',
 'Documentation_Score',
 'Institution_Score']

## Preprocessing for ChromaDB

In [34]:
# Replace NaNs with empty strings to avoid 'nan' in text
df.fillna('', inplace=True)

# Build composite description
df['composite_text'] = (
    "Model: " + df['Model'] + ". " +
    "Task: " + df['Model Task'] + ". " +
    "Action: " + df['Action'] + ". " +
    "Input Data: " + df['Input Data'] + ". " +
    "Framework: " + df['Framework/Library'] + ". " +
    "Institution: " + df['Institution']
)

# Save preprocessed file for ChromaDB ingestion
df[['Model', 'composite_text']].to_csv('prepropessed data/model_metadata_chromadb.csv', index=False)


/var/folders/tw/5bglt3nx2rvbjh_wng4tjm540000gn/T/ipykernel_29158/2161716252.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


In [ ]:
#VERIFYING ACCESS TO OPENAI API MODELS

import openai

client = openai.OpenAI(api_key="YOUR_API_KEY")

response = client.models.list()
for model in response.data:
    print(model.id)


gpt-4o-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
text-embedding-3-small
gpt-4.1-nano
gpt-4.1-nano-2025-04-14
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview
babbage-002
gpt-4
text-embedding-ada-002
chatgpt-4o-latest
gpt-4o-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview
gpt-4o-audio-preview
o1-preview-2024-09-12
gpt-4o-mini-realtime-preview
gpt-4.1-mini
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct-0914
gpt-4o-mini-search-preview
gpt-4.1-mini-2025-04-14
davinci-002
gpt-3.5-turbo-1106
gpt-4o-search-preview
gpt-4-turbo
gpt-3.5-turbo-instruct
gpt-3.5-turbo
gpt-4-turbo-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4-0125-preview
gpt-4o-2024-11-20
whisper-1
gpt-4o-2024-05-13
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-16k
gpt-image-1
o1-preview
gpt-4-0613
text-embedding-3-large
gpt-4o-mini-tts
gpt-4o-transcribe
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-mini-transcribe
gpt-4o-search-preview-2025-03-11
omni-moderation-202